In [63]:
# Aim: Obtains the turnover/lodgement count and amount of a retail loan customer 12 months before the latest loan date

In [64]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
from functools import partial
from dateutil import relativedelta
import dateutil.relativedelta
import matplotlib.pyplot as plt

from pandas.io.parsers import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE                   # For Oversampling
# from outliers import smirnov_grubbs as grubbs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.neural_network import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC

import seaborn as sns
sns.set()

path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
import utils
import pickle

#set working directory
dirr=r"N:\10. Executive\Original\3. 2019\Credit Scoring v3\PyOutput\deploy"
os.chdir(dirr)

In [65]:
# db credentials authentication

In [66]:
conn_string = utils.db_connect()

Enter username: data_analyst
Enter Password:········
Press 'y' to continue, 'c' to cancel or escape:y
Testing Connection...
Connection Succesful!


In [67]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [68]:
cols_selected = pickle.load(open(r"N:\10. Executive\Original\3. 2019\Credit Scoring v2\PyOutput\modeling\cols_selected.sav","rb"))

In [69]:
cols_selected

Index([u'CUS_GENDER', u'PROF_CODE', u'STATE', u'HAS_REACTIVATED',
       u'LOAN_MTH_OF_YEAR', u'BANK_YRS_BIN', u'CUS_AGE_AT_LOAN_BIN',
       u'HAS_ACTIVE_LOAN', u'OUTSTANDING_DEBT_BIN', u'NBR_DISTINCT_DEP_BIN',
       u'HAS_DISTINCT_CALL_TIME_TBILLS', u'HAS_DOM_ACCS',
       u'AVG_BAL_CRNT_BAL_RATIO_BIN', u'AVG_LOANS_MONTH_DIFF_BIN',
       u'AVG_ALL_PRODS_MONTH_DIFF_BIN', u'SUM_LODGEMENT_AMOUNT_BIN',
       u'IN_OUT_AMT_RATIO_BIN', u'IN_OUT_CNT_RATIO_BIN',
       u'COV_LODGEMENT_AMT_BIN', u'COV_LODGEMENT_CNT_BIN'],
      dtype='object')

In [70]:
dataset = pd.read_csv('df_loan_data_basic_var.csv')

In [71]:
len(dataset[cols_selected].columns)

20

In [72]:
dataset.head()

,BRA_CODE,CUS_NUM,CUS_GENDER,PROF_CODE,STATE,CUS_AGE_AT_LOAN_BIN,HAS_ACTIVE_LOAN,OUTSTANDING_DEBT_BIN,NBR_DISTINCT_DEP_BIN,HAS_DISTINCT_CALL_TIME_TBILLS,...,AVG_ALL_PRODS_MONTH_DIFF_BIN,SUM_LODGEMENT_AMOUNT_BIN,IN_OUT_AMT_RATIO_BIN,IN_OUT_CNT_RATIO_BIN,COV_LODGEMENT_AMT_BIN,COV_LODGEMENT_CNT_BIN,BANK_YRS_BIN,LOAN_MTH_OF_YEAR,HAS_DUE_DEBT,HAS_REACTIVATED
0,422,462333,M,99.0,ED,25 - 30,0,< 1000,< 1,0,...,20 - 50,2500000 - 5000000,1 - 1.2,0.2 - 0.3,0.5 - 0.7,0.3 - 0.5,7 - 10,12,0,0
1,251,222783,M,59.0,LA,45 - 60,0,< 1000,< 1,0,...,0 - 10,> 5000000,1 - 1.2,0.2 - 0.3,0.7 - 1,0.5 - 0.7,7 - 10,12,0,1
2,212,790897,M,87.0,LA,45 - 60,0,< 1000,< 1,0,...,< 0,500000 - 1000000,1 - 1.2,0.1 - 0.2,> 1,0.7 - 1,3 - 5,12,0,0
3,245,360682,M,21.0,LA,30 - 35,0,< 1000,< 1,0,...,< 0,< 120000,> 2,0.3 - 0.5,< 0.3,< 0.3,< 1,12,0,0
4,551,549234,F,81.0,AK,25 - 30,0,< 1000,< 1,0,...,< 0,< 120000,< 0.5,< 0.1,> 1,> 1,3 - 5,12,0,0


In [73]:
le = LabelEncoder()
X = dataset[cols_selected].apply(lambda col: le.fit_transform(col)).values

In [74]:
# # Normalization - Using MinMax Scaler
# min_max_scaler = pickle.load(open('min_max_scaler.sav', 'rb'))

In [75]:
# X2 = min_max_scaler.transform(X)
# print min_max_scaler.data_max_

In [76]:
# min_max_scaler = preprocessing.MinMaxScaler()
# min_max_scaler.fit(X)
# X = min_max_scaler.transform(X)

In [77]:
clf_RF = pickle.load(open(r"N:\10. Executive\Original\3. 2019\Credit Scoring v2\PyOutput\modeling\clf_RF.sav", 'rb'))

In [78]:
len(X)

9454397

In [79]:
y_is_good=[]

for ind, sect in enumerate(np.array_split(range(len(X)),100)):
    print ind, ' : ', sect[0], '-',sect[-1]
    y = clf_RF.predict_proba(X[sect])
    y_is_good = y_is_good + [i[1] for i in y]

0  :  0 - 94543


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   15.9s finished


1  :  94544 - 189087


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   15.0s finished


2  :  189088 - 283631


...

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   15.4s finished


98  :  9265311 - 9359853


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   15.5s finished


99  :  9359854 - 9454396


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   18.0s finished


In [80]:
# Check
print len(y_is_good)

9454397


In [81]:
dataset['IS_GOOD_PRED'] = y_is_good

In [84]:
dataset[dataset.IS_GOOD_PRED<0.5]

,BRA_CODE,CUS_NUM,CUS_GENDER,PROF_CODE,STATE,CUS_AGE_AT_LOAN_BIN,HAS_ACTIVE_LOAN,OUTSTANDING_DEBT_BIN,NBR_DISTINCT_DEP_BIN,HAS_DISTINCT_CALL_TIME_TBILLS,...,SUM_LODGEMENT_AMOUNT_BIN,IN_OUT_AMT_RATIO_BIN,IN_OUT_CNT_RATIO_BIN,COV_LODGEMENT_AMT_BIN,COV_LODGEMENT_CNT_BIN,BANK_YRS_BIN,LOAN_MTH_OF_YEAR,HAS_DUE_DEBT,HAS_REACTIVATED,IS_GOOD_PRED
11118,321,279620,M,4.0,FC,30 - 35,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.5 - 0.7,0.7 - 1,0.5 - 0.7,3 - 5,12,0,1,0.495149
16872,208,770730,M,0.0,LA,17 - 25,0,< 1000,< 1,0,...,> 5000000,1 - 1.2,0.2 - 0.3,> 1,> 1,1 - 3,12,0,0,0.489845
30093,394,104976,M,21.0,FC,30 - 35,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.2 - 0.3,0.7 - 1,0.7 - 1,3 - 5,12,0,0,0.464201
36969,327,469597,M,74.0,FC,25 - 30,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.3 - 0.5,0.5 - 0.7,0.5 - 0.7,3 - 5,12,0,0,0.491130
103080,491,446943,M,87.0,OS,35 - 45,0,< 1000,< 1,0,...,> 5000000,1 - 1.2,0.1 - 0.2,0.7 - 1,0.5 - 0.7,> 10,12,0,1,0.491918
104789,721,997526,M,5.0,RV,45 - 60,0,< 1000,< 1,0,...,120000 - 250000,< 0.5,< 0.1,> 1,> 1,7 - 10,12,0,1,0.441171
166418,691,119449,M,16.0,FC,35 - 45,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.3 - 0.5,0.5 - 0.7,0.5 - 0.7,3 - 5,12,0,0,0.473232
179959,319,103302,M,21.0,FC,30 - 35,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.3 - 0.5,0.7 - 1,0.5 - 0.7,3 - 5,12,0,0,0.498204
192505,395,350975,M,6.0,FC,30 - 35,0,< 1000,< 1,0,...,250000 - 500000,1 - 1.2,0.1 - 0.2,0.5 - 0.7,0.5 - 0.7,3 - 5,12,0,0,0.490935
194180,441,336666,M,19.0,KW,45 - 60,0,< 1000,< 1,0,...,> 5000000,1 - 1.2,0.1 - 0.2,0.5 - 0.7,< 0.3,> 10,12,0,1,0.450792


In [82]:
dataset.to_csv('predicted_dataset.csv', index=False)

In [83]:
from sqlalchemy import create_engine

print 'start: ', datetime.now()
conn = create_engine('oracle+cx_oracle://{}:{}@dwdrscan.gtbank.com:1521/?service_name=odidb'.format(un, pwd))
dataset.to_sql('cs_pred_dataset', conn, index=False, if_exists='replace', chunksize=10000, dtype=utils.sqlcol(dataset))
print 'end: ', datetime.now()

start:  2019-12-16 19:58:59.288000


KeyboardInterrupt: 

In [ ]:
# dataset[(dataset.BRA_CODE==205) & (dataset.CUS_NUM==158905)]